# Embudos de producto con secuencia de eventos
El el proyecto de hoy explicaremos las diferencias entre embudos simples y embudos con secuencia de eventos. Como veíamos en el proyecto anterior, con un embudo simple podemos saber el número total de usuarios que llegaron a una etapa del embudo, pero no sabríamos el viaje del cliente (_customer journey_). ¿A qué me refiero? Me refiero a la proporción de clientes que que siguen el camino 'ideal', que en nuestro _dataset_ del día de ayer era `customer_journey` era: `home` → `product_page` → `cart` → `checkout` → `confirmation.` A veces los clientes llegan a la conversión de otras formas y saberlo es valioso. Hoy trabajaremos con un _dataset_ diferente. Comencemos.

## Nuestro datos
Vamos a familiarizarnos.

In [2]:
import pandas as pd
events = pd.read_csv('/datasets/events.csv')
display(events.head(3))

,event_name,event_time,uid
0,pageview,2019-08-20 00:27:30,80419792
1,pageview,2019-01-29 08:11:30,74610198
2,pageview,2019-01-29 01:33:02,101714


Veamos cuántos eventos tenemos.

In [3]:
print(events['event_name'].unique())

['pageview' 'add_to_cart' 'checkout' 'payment']


Tenemos cuatro eventos: `pageview`, `add_to_cart`, `checkout` y `payment`.

## La secuencia de eventos
Para identificar la secuencia de eventos, debemos buscar el momento más temprano en que ocurrió cada evento para cada usuario. Necesitamos pivotear nuestra tabla.

In [4]:
users = events.pivot_table(index = 'uid', columns = 'event_name', values = 'event_time', aggfunc = 'min')
print(users.head())

event_name          add_to_cart             checkout             pageview  \
uid                                                                         
7221        2019-08-04 00:07:52  2019-08-04 00:08:33  2019-08-04 00:07:06   
21371                       NaN                  NaN  2019-02-21 05:35:16   
31608       2019-08-07 21:14:10                  NaN  2019-08-07 21:11:27   
31875                       NaN                  NaN  2019-04-25 03:16:04   
37099                       NaN                  NaN  2019-05-09 22:36:09   

event_name payment  
uid                 
7221           NaN  
21371          NaN  
31608          NaN  
31875          NaN  
37099          NaN  


Lo que acabamos de hacer es acomodar la tabla con base en cada usuario y usar como columnas los valores de `event_name`, es decir, los eventos. Con los eventos como columnas, nuestros registros serán el `event_time`, es decir, el tiempo del evento, pero no cualquier tiempo, sino el valor mínimo (en este caso mínimo es sinónimo de primero.)

Con esta nueva tabla, necesitamos la base de nuestro embudo. Vamos a descubrir el número de visitas al sitio web. 

In [14]:
pageview_count  = users[~users['pageview'].isna()].shape[0]
print('El número de visitantes es:', pageview_count)

El número de visitantes es: 8831


Con este código sabemos cuántos usuarios (`uid`) llegaron a tener al menos un evento `pageview`, que es el evento principal.

Con base en esto, podemos trazar el embudo. Debemos seguir la misma lógica.

In [13]:
step_1 = ~users['pageview'].isna()
step_2 = step_1 & (users['add_to_cart'] > users['pageview'])
step_3 = step_2 & (users['checkout'] > users['add_to_cart'])
step_4 = step_3 & (users['payment'] > users['checkout'])

n_pageview = users[step_1].shape[0]
n_add_to_cart = users[step_2].shape[0]
n_checkout = users[step_3].shape[0]
n_payment = users[step_4].shape[0]

print(f'El número de visitantes es de {n_pageview}.')
print(f'De ellos, agregaron un producto al carrito {n_add_to_cart}.')
print(f'De los que agregaron un producto, iniciaron el pago {n_checkout}.')
print(f'Y finalmente pagaron {n_payment}.')

El número de visitantes es de 8831.
De ellos, agregaron un producto al carrito 2584.
De los que agregaron un producto, iniciaron el pago 354.
Y finalmente pagaron 28.


## Conclusión
Así es como se hace un embudo con base en eventos. Este embudo nos dice cuántos usuarios tomaron la ruta prevista, es de decir: `pageview` → `add_to_cart` → `checkout` → `payment`. Este embudo no sólo cuenta los eventos por usuario, que como dijimos, pueden aparecer más de una vez. De esta forma es más fácil detectar patrones o errores en la página.

¿De qué hablaremos en el próximo cuaderno? No lo sé, primero tengo que solucionar algunos problemas con algunas librerías y sus ubicaciones en mi computadora; errores que surgieron después de querer entrenar modelos a nivel local. Deséenme suerte.